# Bonus HW6: 2 Dimensional Gaussian Copula and Integration

## Preliminaries

### Imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import scipy.stats as stats
import scipy.special as special
import numpy.linalg as linalg

### Random Seed

In [2]:
seed=47693
gen=np.random.default_rng(seed)

### Code

#### Black-Scholes

In [3]:
epsilon=1e-12

def bs_price_fwd(isCall, K, T, F, sigma):
    """ Black's pricing formula
    
    European option  forward price as a function of
    the asset's forward.
    
    :param isCall: True for calls , False for Puts
    :type isCall: Boolean
    :param K: option strike
    :param T: option expiry in years
    :param F: forward of the options underlying asset
    :param sigma: underlying's  volatility
    :return: option's forward price
    """
    ds=np.maximum(0.000001,sigma*np.sqrt(T))
    dsig=0.5*ds*ds
    d2=(np.log(F/np.maximum(K,epsilon))-dsig)/ds
    d1=d2+ds
    if isCall:
        opt= F*special.ndtr(d1) - K*special.ndtr(d2)
    else:
        opt= K*special.ndtr(-d2) - F*special.ndtr(-d1)
    return opt

#### SVI Curve

In [4]:
class SVICurve:
    def __init__(self,T,F,vol_ATM,b,rho,m,sigma):
        self.T=T
        self.F=F
        self.a=vol_ATM**2*T-b*(-rho*m+np.sqrt(m**2+sigma**2)) 
        self.b=b
        self.rho=rho
        self.m=m
        self.sigma=sigma        
    def __call__(self,K):
        k=np.log(K/self.F)
        var=self.a + self.b*(self.rho*(k-self.m)+np.sqrt((k-self.m)**2+self.sigma**2))
        return np.sqrt(var/self.T)

#### RiskNeutral Distribution 

In [5]:
class RiskNeutral:
    def __init__(self,T,F,vol):
        self.T=T
        self.F=F
        self.vol=vol
    def cdf(self,K,dk=0.001):
        dK=K*dk
        K0=K-dK
        sig0=self.vol(K0)
        p0=bs_price_fwd(False,K0,self.T,self.F,sig0)
        K1=K+dK
        sig1=self.vol(K1)
        p1=bs_price_fwd(False,K1,self.T,self.F,sig1)
        return  (p1-p0)/dK/2.0
    def pdf(self,K,dk=0.001):
        dK=K*dk
        return (self.cdf(K)-self.cdf(K-dK))/dK       
    def ppf(self,u,tol=1e-4,max_iter=21): # inverse cdf
        T=self.T
        F=self.F
        sigma=self.vol(F)
        var=sigma**2*T
        std=np.sqrt(var)
        S=np.empty_like(u.ravel())
        
        S0=F*np.exp(-15*std)
        cdf0=self.cdf(S0)
        pdf0=self.pdf(S0)
        u1=u.ravel() #linearize into one long vector
        idx=u1.argsort() # we will visit them in increasing order
        for i1 in range(len(idx)):
            U=u1[idx[i1]]
            # solve equation  
            #    cdf(S)=u
            # using brent's method
            iter=0
            while np.abs(cdf0-U)/cdf0>tol:
                S1=S0+(U-cdf0)/pdf0
                cdf1=self.cdf(S1)
                pdf0=(cdf1-cdf0)/(S1-S0)
                cdf0=cdf1
                S0=S1
                iter+=1
                if (iter>=max_iter):
                    break
            S[idx[i1]]=S0
                    
        return S.reshape(u.shape)
    def rvs(self,gen,size,tol=1e-4,max_iter=21):
        u=gen.uniform(size=size)
        return self.ppf(u,tol,max_iter)

#### GaussianCopula

In [6]:
class GaussianCopula:
    def __init__(self,rho):
        self.rho=rho
    def generate(self,N,gen):
        D=len(self.rho)
        Z=stats.multivariate_normal(np.zeros(D),self.rho,seed=gen,allow_singular=True).rvs(size=N)
        U=stats.norm.cdf(Z)
        return U

#### Generate Copula Correlated Distributions

In [7]:
def generate_distribution(dists,copula,N,gen):
    U=copula.generate(N,gen) # correlated U(0,1)
    Ss=[]
    for i1,dist in enumerate(dists):
        S=dist.ppf(U[:,i1]) # inverse cdf
        Ss.append(S)
    return np.stack(Ss,axis=-1)

#### Two Dimensional Mid Point Rule

In [8]:
def mid_point_rule_2d(func, a1, b1, a2,b2, N):
    dx1=(b1-a1)/(N+1)
    dx2=(b2-a2)/(N+1)
    x1=np.linspace(a1+dx1/2,b1-dx1/2,N)
    x2=np.linspace(a2+dx2/2,b2-dx2/2,N)   
    y=func(x1[:,np.newaxis],x2[np.newaxis,:]) 
    I= np.sum(y)  
    return dx1*dx2*I

#### Analytic Worst of 2 Asset Call/Put Price

In [9]:

def N2(z1,z2,corr):
    return stats.multivariate_normal(mean=[0,0], cov=[[1,corr],[corr,1]]).cdf([z1,z2])

def M(T,K,F1,F2,sig1,sig2,rho):  
    var1=0.5*sig1**2*T
    var2=0.5*sig2**2*T
    std1=sig1*np.sqrt(T)
    std2=sig2*np.sqrt(T)
    gamma1=(np.log(F1/K)-var1)/std1
    gamma2=(np.log(F2/K)-var2)/std2
    sigma_square=(sig1**2)+(sig2**2)-2*rho*sig1*sig2
    sigma=np.sqrt(sigma_square)
    var=0.5*sigma_square*T
    std=sigma*np.sqrt(T)

    alpha1=gamma1+std1
    beta1=(np.log(F2/F1)-var)/std
    theta1=(rho*sig2-sig1)/sigma

    alpha2=gamma2+std2
    beta2=(np.log(F1/F2)-var)/std
    theta2=(rho*sig1-sig2)/sigma
    
    alpha3=gamma1
    beta3=gamma2
    theta3=rho
    

    N2_cdf_1=N2(alpha1,beta1,theta1)
    N2_cdf_2=N2(alpha2,beta2,theta2)
    N2_cdf_K=N2(alpha3,beta3,theta3)
   
    M_cdf=(F1*N2_cdf_1)+(F2*N2_cdf_2)-(K*N2_cdf_K)
    
    return M_cdf

# analytic worst of two option
def worst_option_fwd(is_call,T,K,F1,F2,sig1,sig2,rho):  
    if is_call==True:
        payoff=M(T,K, F1,F2,sig1,sig2,rho)
    else:
        payoff=K-M(T,0.00001,F2,F1,sig2,sig1,rho)+M(T,K,F2,F1,sig2,sig1,rho)        
    return payoff

### Problem 1 Cholesky Decomposition in Two Dimensions

#### Problem 1.1
using [`numpy.linalg.cholesky`](https://numpy.org/doc/stable/reference/generated/numpy.linalg.cholesky.html)
compute the cholesky decomposition $L$ of two assets with correlation $\rho=70\%$

In [10]:
rho=0.7

In [11]:
corr=np.array([[1,rho],[rho,1]])
corr

array([[1. , 0.7],
       [0.7, 1. ]])

In [12]:
L=linalg.cholesky(corr)
L

array([[1.        , 0.        ],
       [0.7       , 0.71414284]])

#### Problem 1.2

Check that the cholesky decomposition $L$ is given by:

$$
    L = \begin{pmatrix}
            1 & 0 \\
            \rho & \sqrt{1-\rho^2}
        \end{pmatrix}
$$

[NOTE] 

This implies that, given independent $\mathcal{N}(0,1)$ random variables $z_1$ and $z_2$ we can generate
\begin{align}
    w_1 &= z_1 \\
    w_2 &= \rho z_1 + \sqrt{1-\rho^2} z_2
\end{align}
Where $w_1$ and $w_2$ are again Gaussain $\mathcal{N}(0,1)$ but with correlation $\rho$.


In [13]:
np.sqrt(1-rho**2)

0.714142842854285

## Problem 2: 2 Dimensional Gaussian Copula Transform:

We can transform two undependent $U(0,1)$ random variables $u_1$, $u_2$ into two $U(0,1)$ correlated variables $v_1, v_2$ using the Gaussian copula with the function:
$$
    (v_1,v_2) = C_T(u_1,u_2;\rho)
$$
that is implemented with following algorithm:
\begin{align*}
    z_1&=\text{N}^{-1}(u_1) & z_1,z_2 \ \ \text{are independent Gaussians}\\
    z_2&=\text{N}^{-1}(u_2) &\\
    w_1 &= z_1  & w_1,w_2 \ \ \text{are correlated Gaussians}\\
    w_2 &= \rho z_1 + \sqrt{1-\rho^2} z_2 \\
    v_1 &= \text{N}(w_1) & v_1, v_2\ \ \text{are correlated U}(0,1) \\
    v_2 &= \text{N}(w_2)
\end{align*}

where $N(z)$ is the CDF for the standard Gaussian $\mathcal{N}(0,1)$ distribution and $N^{-1}(u)$ is its  inverse.

Write function `gaussian_copula_transform` implementing the transformation above:

[HINT]  In `scipy` the Gaussian  CDF $N(z)$ is implemented by function [`scipy.special.ndtr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy_special.ndtr.html), and its inverse $N^{-1}(z)$ is implemented by [`scipy.special.ndtri`](https://docs.scipy.org/doc/scipy/reference/generated/scipy_special.ndtri.html)

In [14]:
def gaussian_copula_transform(u1,u2,rho):
    pass

In [15]:
def gaussian_copula_transform(u1,u2,rho):
    z1=special.ndtri(u1)
    z2=special.ndtri(u2)
    w1=z1
    w2=rho*z1+np.sqrt(1-rho**2)*z2
    x1=u1
    x2=special.ndtr(w2)
    return x1,x2 # correlated U(1,0) marginals

## Problem 3: Analytic Wost Of Call Price

Compute the **analytic price** of a **Worst of 2 assets** call option with
1. Expiry $T=1$ year.
2. Strike $K=0.8$
using the function   [worst_option_fwd](#analytic-worst-of-2-asset-callput-price) that is implemented above in the [Code](#code) section.

Given that the assets have **correlation** $\rho=70\%$
and the following characteristics:
1. **Asset 1**: Forward $F=1$ and volatility $\sigma_{\text{ATM},1}=19\%$
2. **Asset 2**: Forward $F=1$ and volatility $\sigma_{\text{ATM},1}=25\%$



In [16]:
T=1
is_call=True
K=0.8

In [17]:
F1=1
sigma_ATM=0.19
F2=1
sigma_ATM2=0.25

rho=0.7


In [18]:
I0=worst_option_fwd(is_call,T,K,1,1,sigma_ATM,sigma_ATM2,rho)
I0

0.15491236163669841

## Problem 4: Log Normal Price

### Problem 4.1
Generate two [`scipy.stats.lognorm`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.lognorm.html) distributions based on the market date of 
Problem 3

In [19]:
var1=sigma_ATM**2*T
dist1_ln=stats.lognorm(s=np.sqrt(var1),scale=F1*np.exp(-0.5*var1))

In [20]:
var2=sigma_ATM2**2*T
f2=1
dist2_ln=stats.lognorm(s=np.sqrt(var2),scale=F2*np.exp(-0.5*var2))

### Problem 4.2:

Write the worst of two options payout
$$
    \text{payout}_{W}(S1,S2,K) = \max ( \min(S1,S2)-K,0)
$$

In [21]:
def worstof_call_payoff(S1,S2,K):
    W=np.minimum(S1,S2)  # worst of price
    return np.maximum(W-K,0)


### Problem 4.3:

Write the **Gaussian copula** transformed payout using the following algorithm:
\begin{align*}
    v_1,v_2&=C_T(u_1,u_2,\rho) \\
    S_1 &= CDF_1^{-1}(v_1) \\
    S_2 &= CDF_2^{-1}(v_2) \\
    \text{payout}&=\text{payout}_{W}(S1,S2,K)
\end{align*}
where:
1. $C_T(u_1,u_2,\rho)$ is the two dimensional Gaussian copula transformation `gaussian_copula_transform` implemented in Problem 2.
2. $CDF_1^{-1}$ is the inverse CDF of asset 1.
3. $CDF_2^{-1}$ is the inverse CDF of asset 2.
4. $\text{payout}_{W}(S1,S2,K)$ is the worst of two assets call payout implemented in Problem 4.2

[HINT] For `scipy.stats` distributions the inverse CDF is called `ppf`

In [22]:
def worstof_option_payoff_transformed(u1,u2,K,dist1,dist2,rho):
    pass

In [23]:
def worstof_option_payoff_transformed(u1,u2,K,dist1,dist2,rho):
    v1,v2=gaussian_copula_transform(u1,u2,rho)
    S1=dist1.ppf(v1)
    S2=dist2.ppf(v2)
    return worstof_call_payoff(S1,S2,K)

### Problem 4.4:

Use the [mid_point_rule_2d](#two-dimensional-mid-point-rule) to integrate the worst of two call transformed payout you implemented in Problem 4.3.

Compare the result to the Analytic Result.

[HINTS] 
1. The integration limits are $(0,1)$ for both $u_1$ and $u_2$.
2. Choose a number of points that you think is sufficient to provide $\approx 1\%$ accuracy

In [24]:
N_g=300
I_g=mid_point_rule_2d(lambda u1,u2: worstof_option_payoff_transformed(u1,u2,K,dist1_ln,dist2_ln,rho),0,1,0,1,N_g)
I_g

0.15379132613338017

In [25]:
(I_g-I0)/I0

-0.007236578743453031

## Problem 5: Risk Neutral distribution

From option market data we calibrate volatility curve for $T=1$ year two both assets.

Both assets have unit forwards ($F_1=1$ and $F_2=1$) and the following SVI curve parameters:
1. **Asset 1**:
    1. $\sigma_{ATM}$=19%
    1. $b$=0.13
    2. $\rho$ = -0.734
    3. $m$=0.128
    4. $\sigma$=0.118
1. **Asset 2**:
    1. $\sigma_{ATM}$=25%
    1. $b$=0.07
    2. $\rho$ = -0.85
    3. $m$=0.08
    4. $\sigma$=0.13

Define the risk neutral distributions of both assets

In [26]:
T=1
F=1
sigma_ATM=0.19
b=0.13
rho_svi=-0.734
m=0.128
sigma=0.118

In [27]:
F2=1
sigma_ATM2=0.25
b2=0.07
rho2_svi=-0.85
m2=0.08
sigma2=0.13

In [28]:
vol1=SVICurve(T,F,sigma_ATM,b,rho_svi,m,sigma)
dist1=RiskNeutral(T,F,vol1)

In [29]:
vol2=SVICurve(T,F2,sigma_ATM2,b2,rho2_svi,m2,sigma2)
dist2=RiskNeutral(T,F2,vol2)

### Problem 5.2

Price the worst of call on the works of the two assets (by integration, as you did in Problem 4) using the risk neutral distributions defined in Problem 5.1 and a correlation $\rho=70\%$ between the two assets.

In [30]:
I=mid_point_rule_2d(lambda u1,u2: worstof_option_payoff_transformed(u1,u2,K,dist1,dist2,rho),0,1,0,1,300)
I

0.17293261104589433

### Problem 5.3

We have no analytic formula to check the accuracy of the price computed in Problem 5.2

Use Monte Carlo to generate $N=10,000$ samples of the risk neutral distribution of the two assets, correlated with the Gaussian Copula.

Compare the prices you obtained to those of problem 5.2

In [31]:
N=10_000
copula=GaussianCopula(corr)
S=generate_distribution([dist1,dist2],copula,N,gen)

In [32]:
pay=worstof_call_payoff(S[:,0],S[:,1],K)
I_mc=pay.mean()
dI=pay.std()/np.sqrt(N)
I_mc,dI

(0.1744546010131932, 0.0014293983664954068)

In [33]:
# the two prices are comparable given the MC standard deviation
(I-I_mc)/dI

-1.0647766241894414